In [1]:
import numpy as np

def getMat(iArr, matrix):
    n = len(iArr)
    new_mat = np.ndarray(dtype="double", shape=(n,n))
    for i in range(n):
        for j in range(n):
            new_mat[i][j] = matrix[iArr[i]][iArr[j]]

    return new_mat

def binary(n, digits):
    binary = str("{0:b}".format(n))
    return (digits-len(binary))*"0"+binary

def contains(a, b):
    for i in range(len(a)):
        if(a[i]=='1' and b[i]=='0'): return False
    return True


def find_route(matrix, locations, time_limit, f):
    bads = []

    best_value = -1
    best_order = []

    n = len(locations)
    for i in range(1, 2**n):
        if(i%100000==0): print(i)

        bin = binary(i, n)

        cont = True
        for j in range(len(bads)):
            if(contains(bads[j], bin)):
                cont = False
                break

        if(cont):
            iArr, posArr = [], []

            for j in range(n):
                if(bin[j]=='1'): iArr.append(j)

            time, order = optimize2(getMat(iArr, matrix))
            for i in range(len(iArr)): posArr.append(locations[iArr[order[i]]])
            if(time<=time_limit and f(posArr)>best_value): best_value, best_order = f(posArr), posArr
            elif(time>time_limit): bads.append(bin)

    return best_order

def optimize(matrix):
    n, time, order = len(matrix[0]), 0, [0]
    for i in range(n-1):
        time += matrix[i][i+1]
        order.append(i+1)
    time += matrix[n-1][0]
    return time, order

def optimize2(matrix):
    n, max_dist, loc = len(matrix[0]), 0, -1
    for i in range(0, n-1):
        for j in range(i+1, n):
            if(matrix[i][j]>max_dist):
                max_dist, loc = matrix[i][j], i
    
    time, order, dist = 0, [loc], 0
    
    for i in range(0, n-1):
        min_dist, loc = 1e10, -1
        for j in range(n):
            if(not j in set(order)):
                dist = matrix[order[-1]][j]
                if(dist<min_dist): min_dist, loc = dist, j

        order.append(loc)
        time+= min_dist

    time += matrix[order[0]][order[-1]]
    return time, order

#rating parameters
[a, b] = [2e-3, 0.5]

def metric_func(posArr):
    n, sum = len(posArr), 0
    for i in range(0, n): 
        sum += posArr[i].rating + a*(posArr[i].reviews)**b
    return sum/n**(0.5)

def metric_func_2(posArr):
    n, sum = len(posArr), 0
    for i in range(0, n): sum += posArr[i].reviews
    return sum

In [2]:
import numpy as np
import googlemaps
import datetime

KEY_file = open('key.txt', 'r')
KEY=KEY_file.readlines()[0]

def getDistance(lat1, lon1, lat2, lon2):
    dist = ((lat1-lat2)**2+(lon1-lon2)**2)**0.5
    return dist*69/3*3600 #conversion form lat/lon to miles to hours to seconds

def adjMatrixApprox(places):
    n = len(places)
    matrix = np.zeros(shape=(n, n), dtype='double')

    for i in range(0,n):
        for j in range(0,n):
            if(i==j): matrix[i][j] = 0
            elif(i>j): matrix[i][j] = matrix[j][i]
            else: matrix[i][j] = int(getDistance(places[i].lat, places[i].lon, places[j].lat, places[j].lon))

    return matrix

def adjMatrixReal(places):
    n = len(places)
    matrix = np.ndarray(dtype="double", shape=(n,n))
    now=datetime.datetime.now()

    coords = np.zeros(shape=(n,2), dtype='double')
    for i in range(n):
        coords[i]= [places[i].lat, places[i].lon]

    gmaps_real = googlemaps.Client(key=KEY)
    gmaps_matrix = gmaps_real.distance_matrix(coords, coords,
                        mode="walking",
                        avoid="ferries",
                        departure_time=now)

    for i in range(0,n):
        for j in range(0,n):
            if(i==j): matrix[i][j] = 0
            else: matrix[i][j] = gmaps_matrix['rows'][i]['elements'][j]['duration']['value']

    return matrix

def printMatrix(matrix):
    n = matrix.shape[0]
    for i in range(0, n):
        for j in range(0, n):
            print(matrix[i][j], end=" ")
        print()
    print()

In [3]:
from googlesearch import search
import re

import requests
from bs4 import BeautifulSoup

pause = 0.0

def GS(query, count=10):
    results = []
    for i in search(query=query, tld='co.in', lang='en',
            num=count, stop=count, pause=pause):
        results.append(i)
    return results

def GS_site(query, site, count=10):
    results = GS(query, count)
    out = []

    for i in range(len(results)):
        if re.search(site, results[i]) != None:
            out.append(results[i])

    return out

def printArr(results):
    for i in range(len(results)):
        print(results[i])

def getHTML(URL):
    r = requests.get(URL)
    soup = BeautifulSoup(r.content, 'html5lib')
    return soup

def getHTML_all(URLs):
    soups = []
    for URL in URLs:
        soups.append(BeautifulSoup(requests.get(URL).content, 'html5lib'))

    return soups

def GS_HTML(query):
    results = GS(query, 1)
    return getHTML(results[0])

def GS_HTML_site(query, site):
    results = GS_site(query, site, 5)
    return getHTML(results[0])

def GS_HTML_all(query):
    results = GS(query, 10)
    return getHTML_all(results)

def GS_HTML_site_all(query, site):
    results = GS_site(query, site, 10)
    return getHTML_all(results)

def getElements(html, object, attrs):
    arr = []
    for obj in html.find_all(object, attrs=attrs):
        arr.append(obj.text)
    return arr

def getElements_all(htmls, object, attrs):
    unique = set([])
    for html in htmls:
        for obj in html.find_all(object, attrs=attrs):
            unique.add(obj.text)

    arr = []
    for i in unique:
        arr.append(i)

    return arr

def GS_elements(query, object, attrs):
    html = GS_HTML(query)
    return getElements(html, object, attrs)

def GS_elements_site(query, site, object, attrs):
    html = GS_HTML_site(query, site)
    return getElements(html, object, attrs)

def GS_elements_all(query, objects, attrs):
    htmls = GS_HTML_all(query)
    return getElements_all(htmls, object, attrs)

def GS_elements_site_all(query, site, object, attrs):
    htmls = GS_HTML_site_all(query, site)
    return getElements_all(htmls, object, attrs)

def tripAdvisorTourism(city):
    query = "TripAdvisor" + city
    site = "https://www.tripadvisor.com/Tourism"
    object = "span"
    attrs = {'class', 'social-shelf-items-ShelfLocationSection__name--CdA_A'}

    places = GS_elements_site_all(query, site, object, attrs)
    return places

def tripAdvisorAttractions(city):
    query = "TripAdvisor" + city
    site = "https://www.tripadvisor.com/Attractions"
    object = "a"
    attrs = {'class', 'attractions-attraction-overview-pois-PoiInfo__name--SJ0a4'}

    places = GS_elements_site_all(query, site, object, attrs)
    return places

def tripAdvisorCategory(city, category):
    query = "TripAdvisor" + city + " " + category
    site = "https://www.tripadvisor.com/Attractions"
    object = "div"
    attrs = {'class', 'listing_title'}

    places = GS_elements_site_all(query, site, object, attrs)

    new_places = []
    for place in places:
        try:
            i = 0
            while(place.split("\n")[i]==""): i+=1
            new_places.append(place.split("\n")[i])
        except IndexError: pass

    return new_places

def tripAdvisor(city):
    unique = set([])
    places = tripAdvisorTourism(city)
    for place in places: unique.add(place)
    #print(len(places))

    places = tripAdvisorAttractions(city)
    for place in places: unique.add(place)
    #print(len(places))

    categories = ["Sights and Landmarks", "Museums", "Architectural Buildings",
        "Historic Sites", "Monuments and Statues", "Parks"]

    for category in categories:
        places = tripAdvisorCategory(city, category)
        for place in places: unique.add(place)
        #print(len(places))

    print("Location Candidates: " + str(len(list(unique))))
    return list(unique)

def tripAdvisorQuick(city):
    unique = set([])
    places = tripAdvisorTourism(city)
    for place in places: unique.add(place)
    #print(len(places))

    places = tripAdvisorAttractions(city)
    for place in places: unique.add(place)
    #print(len(places))

    print("Location Candidates: " + str(len(list(unique))))
    return list(unique)

In [4]:
import googlemaps

f = open("key.txt", "r")
KEY = f.readlines()[0]
gmaps_real = googlemaps.Client(key=KEY)

def getCity(city):
    data =gmaps_real.places(city)
    lat = data['results'][0]['geometry']['location']['lat']
    lon = data['results'][0]['geometry']['location']['lng']
    return [lat, lon]

def getData(location, city):
    data = gmaps_real.places(location, location=city)
    if(data['status']=='ZERO_RESULTS'): return False

    try:
        lat = data['results'][0]['geometry']['location']['lat']
        lon = data['results'][0]['geometry']['location']['lng']
        address = data['results'][0]['formatted_address']
        name = data['results'][0]['name']
        rating = data['results'][0]['rating']
        reviews = data['results'][0]['user_ratings_total']
        types = data['results'][0]['types']
        
        return [lat, lon, address, name, rating, reviews, types]

    except KeyError: return False

def solve(locations):
    start = [locations[0].lat, locations[0].lon]
    end = [locations[-1].lat, locations[-1].lon]

    waypoints = []
    for i in range(1, len(locations)-1):
        waypoint = [locations[i].lat, locations[i].lon]
        waypoints.append(waypoint)


    directions = gmaps_real.directions(origin=start,
                            destination=end,
                            mode="walking",
                            waypoints=waypoints,
                            avoid="ferries",
                            optimize_waypoints=True)

    order, distances, durations = [locations[0]], [], []
    waypoint_order = directions[0]['waypoint_order']
    for waypoint in waypoint_order: order.append(locations[waypoint+1])
    order.append(locations[-1])

    for i in range(len(directions[0]['legs'])):
        distances.append(directions[0]['legs'][i]['distance']['value'])
        durations.append(directions[0]['legs'][i]['duration']['value'])

    return order, distances, durations


In [5]:
import google

bad = ['atm', 'bar', 'beauty_salon', 'bicycle_store', 'bus_station',
    'car_dealer', 'car_rental', 'car_repair', 'car_wash', 'dentist',
    'doctor', 'drugstore', 'electrician', 'funeral_home', 'gas_station',
    'hair_care', 'insurance_agency', 'laundry', 'liquor_store', 'locksmith',
    'lodging', 'movie_rental', 'moving_company', 'night_club', 'painter',
    'pharmacy', 'physiotherapist', 'plumber', 'post_office',
    'real_estate_agency', 'roofing_contractor', 'rv_park', 'storage',
    'taxi_stand', 'veterinary_care']

park = ['park']

restaurant = ['meal_delivery', 'meal_takeaway', 'restaurant']

class Place:
    filled = False

    def __init__(self, start, location, city):
        self.city_coords = city
        self.filled = self.update(location, city)

    def update(self, location, city):
        data = getData(location, city)
        if(data==False): return False
        self.lat, self.lon, self.address, self.name, self.rating, self.reviews, self.types = data
        return True

    def print(self):
        print(self.name)
        print(self.address)
        print("("+str(self.lat)+", "+str(self.lon)+")")
        print(str(self.rating)+" ("+str(self.reviews)+")")
        print(self.types)
        print()

    def quickPrint(self):
        print(str(self.name) + " - " + str(self.rating)[:4] + " (" + str(self.reviews) + ")")

    def isBad(self):
        if(getDistance(self.city_coords[0], self.city_coords[1], self.lat, self.lon)>72000): return True
        if(self.rating<3 or self.reviews<25): return True
        return len(list(set(bad) & set(self.types)))>0

    def isPark(self):
        return len(list(set(park) & set(self.types)))>0

    def isRestaurant(self):
        return len(list(set(restaurant) & set(self.types)))>0

def addPlaces(strs, city):
    names = set([])
    places = []

    city_coords = getCity(city)

    for place in strs:
        if place != "":
            temp = Place(False, place, city_coords)
            if(temp.filled==True):
                if(len(set([temp.address])&names)==0 and temp.isBad()==False):
                    places.append(temp)
                    names.add(temp.address)

    return places

def printPlaces(places):
    for place in places:
        place.print()

def quickPrintPlaces(places):
    for place in places:
        place.quickPrint()

def sortPlaces_reviews(places):
    places.sort(key=lambda x: x.reviews, reverse=True)

def sortPlaces_rating(places):
    places.sort(key=lambda x: x.rating, reverse=True)

def getPlaces(city, start=True):
    strs = tripAdvisor(city)
    places = addPlaces(strs, city)
    sortPlaces_reviews(places)
    return __cleanPlaces__(places, city, start)

def getPlacesQuick(city, start=True):
    strs = tripAdvisorQuick(city)
    places = addPlaces(strs, city)
    sortPlaces_reviews(places)
    return __cleanPlaces__(places, city, start)

def __cleanPlaces__(places, city, start):
    [parks, parks_max] = [0, 3]
    [rests, rests_max] = [0, 3]

    locs = []
    new_places = []

    for place in places:

        cont = True
        for loc in locs:
            if(getDistance(loc[0], loc[1], place.lat, place.lon)<120):
                cont = False
                break

        if(cont):
            if(place.isPark() and parks<parks_max):
                new_places.append(place)
                locs.append([place.lat, place.lon])
                parks+=1

            elif(place.isRestaurant() and rests<rests_max):
                new_places.append(place)
                locs.append([place.lat, place.lon])
                rests+=1

            elif(not place.isPark() and not place.isRestaurant()):
                new_places.append(place)
                locs.append([place.lat, place.lon])

    print("Acceptable Locations: " + str(len(new_places)))
    print()
    
    places = new_places[:min(20,len(new_places))]
    sortPlaces_rating(places)
    return places


In [6]:
import gmaps
gmaps.configure(api_key=KEY)

def getFig(order):
    fig = gmaps.figure()

    start = [order[0].lat, order[0].lon]
    end = [order[-1].lat, order[-1].lon]

    waypoints = []
    for i in range(1, len(order)-1):
        waypoint = [order[i].lat, order[i].lon]
        waypoints.append(waypoint)

    layer = gmaps.directions.Directions(start, end, waypoints, mode='walking')

    fig.add_layer(layer)
    return fig

In [7]:
def main1(city):
    places = getPlaces(city)
    quickPrintPlaces(places)
    print()
    
    matrix_approx = adjMatrixApprox(places)
    return places, matrix_approx

In [8]:
def main2(places, matrix_approx, time_limit):
    order = find_route(matrix_approx, places, time_limit, metric_func)
    order, distances, durations = solve(order)
    
    print()
    for i in range(len(distances)):
        order[i].quickPrint()
        print("Walk " + str(distances[i]) + " feet for " + str(durations[i]) + " seconds to the next stop\n")
    order[-1].quickPrint()
    
    fig = getFig(order)
    return fig, order

In [19]:
city = "Nashville"
time_limit = 3600

In [20]:
places, matrix_approx = main1(city)

Location Candidates: 176
Acceptable Locations: 66

Grand Ole Opry - 4.8 (12186)
Ryman Auditorium - 4.8 (8807)
Monell's - 4.8 (3462)
Andrew Jackson’s Hermitage - 4.7 (2772)
Frist Art Museum - 4.7 (2651)
Tennessee Performing Arts Center - 4.7 (2062)
Nashville Zoo at Grassmere - 4.6 (10118)
Country Music Hall of Fame and Museum - 4.6 (8534)
Hattie B's Hot Chicken - Midtown Nashville, TN - 4.6 (6870)
The Johnny Cash Museum & Cafe - 4.6 (5578)
The Parthenon - 4.6 (4821)
Music City Center - 4.6 (3632)
Adventure Science Center - 4.6 (3626)
Belle Meade Plantation - 4.6 (2134)
Opry Mills - 4.5 (18702)
Nissan Stadium - 4.5 (6836)
Nashville Farmers' Market - 4.5 (4540)
Cooter's Nashville - 4.5 (2424)
Biscuit Love Gulch - 4.4 (3756)
The Mall at Green Hills - 4.3 (4425)



In [21]:
fig, order = main2(places, matrix_approx, time_limit)

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000

Tennessee Performing Arts Center - 4.7 (2062)
Walk 653 feet for 461 seconds to the next stop

Ryman Auditorium - 4.8 (8807)
Walk 335 feet for 259 seconds to the next stop

The Johnny Cash Museum & Cafe - 4.6 (5578)
Walk 425 feet for 345 seconds to the next stop

Country Music Hall of Fame and Museum - 4.6 (8534)
Walk 409 feet for 339 seconds to the next stop

Music City Center - 4.6 (3632)
Walk 1160 feet for 945 seconds to the next stop

Frist Art Museum - 4.7 (2651)
Walk 868 feet for 685 seconds to the next stop

Biscuit Love Gulch - 4.4 (3756)


In [22]:
fig

Figure(layout=FigureLayout(height='420px'))

In [23]:
for i in range(len(order)): print(chr(ord('A')+i) + " - " + str(order[i].name))

A - Tennessee Performing Arts Center
B - Ryman Auditorium
C - The Johnny Cash Museum & Cafe
D - Country Music Hall of Fame and Museum
E - Music City Center
F - Frist Art Museum
G - Biscuit Love Gulch
